#Titanic - Machine Learning from a Disaster


In [84]:
import pandas
import numpy
from sklearn import neighbors
from sklearn.neighbors import DistanceMetric 
from pprint import pprint

MY_TITANIC_TRAIN ='./sample_data/train.csv'
MY_TITANIC_TEST = './sample_data/test.csv'
titanic_dataframe = pandas.read_csv(MY_TITANIC_TRAIN)
print('length: {0} '.format(len(titanic_dataframe)))
titanic_dataframe.head(2)






length: 891 


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [22]:
ave_age = 29.699118
titanic_dataframe['Age'].fillna(ave_age, inplace=True)





,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


We replaced the NA age values with the mean ages to complete data set.

In [ ]:
titanic_dataframe = titanic_dataframe.dropna()

titanic_dataframe['Sex'].unique()
titanic_dataframe['Gender'] = titanic_dataframe['Sex'].map({'female': 0, 'male': 1}).astype(int)
titanic_dataframe['Embarked'].unique()
titanic_dataframe['Port'] = titanic_dataframe['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)

titanic_dataframe.drop(['Sex', 'Embarked'], axis=1, inplace=True)

In [69]:
titanic_dataframe.head(0)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Gender,Port
0,1,0,3,22.0,1,0,7.25,1,2


In [ ]:
def hamming_distance(input_row, train_row):
    distance = []
    for t_col, i_col in zip(train_row, input_row):
        if t_col != i_col:
            distance.append(1)
        else:
            distance.append(0)

    return sum(distance)

factors = [ 'Pclass', 'Port', 'Gender']
hamming = DistanceMetric.get_metric('hamming')

In [66]:
def euclidian_distance(row, train_row):
    distance = list((row[i] - train_row[i])^2
                     for i in range(len(input_row)) if isnumeric(row[i]) & isnumeric(train_row[i])
               )
    return sqrt(sum(distance))

numericals = ['Age', 'SibSp', 'Parch', 'Fare']
euclidian = DistanceMetric.get_metric('euclidean')

In [83]:
def knn(row, train, k):
    distance = []
    for train_row in train:
        hamming = hamming_distance(row, train_row[factors])
        euclidian  = euclidian_distance(row, train_row[numericals])
        distance.append(hamming + euclidian)
    distance.sort(key=operator.itemgetter(0))
    out = distance[ :k]
    return (row[id], 1 if sum(out) > k//2 else 0)

who_dies = []
for row in titanic_dataframe:
    print(knn(row, titanic_dataframe, 10))
# knn(titanic_dataframe.iterrows(), titanic_dataframe, 10)


TypeError: string indices must be integers